In [99]:
import numpy as np
import pandas as pd
import nltk
import spacy
import re
import string

In [100]:
df = pd.read_csv("datasets/imdb.csv")
print(type(df))

df['review'] = df['review'].astype(str)

df.head()

#Deixando lower case
df['review_lower'] = df['review'].str.lower()

df.head()

<class 'pandas.core.frame.DataFrame'>


,review,sentiment,review_lower
0,One of the other reviewers has mentioned that ...,positive,one of the other reviewers has mentioned that ...
1,A wonderful little production. <br /><br />The...,positive,a wonderful little production. <br /><br />the...
2,I thought this was a wonderful way to spend ti...,positive,i thought this was a wonderful way to spend ti...
3,Basically there's a family where a little boy ...,negative,basically there's a family where a little boy ...
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,"petter mattei's ""love in the time of money"" is..."


In [110]:
punctuation = string.punctuation
#help(str)
#Removendo pontuação
def remove_punctuation(text):
    return text.translate(str.maketrans("", "", punctuation))

#removendo HTMLS
PADRAO_TAG = "<.*?>"
def clean_html(text, padrao=PADRAO_TAG):
    return re.sub(padrao, "", text)
    
df['review_lower'] = df['review_lower']\
    .apply(lambda text: clean_html(text))

df['review_sem_pont'] = df['review_lower']\
    .apply(lambda text: remove_punctuation(text))

df.head()


,review,sentiment,review_lower,review_sem_pont,review_stopwords,review_stemmed
0,One of the other reviewers has mentioned that ...,positive,one of the other reviewers has mentioned that ...,one of the other reviewers has mentioned that ...,one reviewers mentioned watching 1 oz episode ...,one review mention watch 1 oz episod youll hoo...
1,A wonderful little production. <br /><br />The...,positive,a wonderful little production. the filming tec...,a wonderful little production the filming tech...,wonderful little production filming technique ...,wonder littl product film techniqu unassum old...
2,I thought this was a wonderful way to spend ti...,positive,i thought this was a wonderful way to spend ti...,i thought this was a wonderful way to spend ti...,thought wonderful way spend time hot summer we...,thought wonder way spend time hot summer weeke...
3,Basically there's a family where a little boy ...,negative,basically there's a family where a little boy ...,basically theres a family where a little boy j...,basically theres family little boy jake thinks...,basic there famili littl boy jake think there ...
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,"petter mattei's ""love in the time of money"" is...",petter matteis love in the time of money is a ...,petter matteis love time money visually stunni...,petter mattei love time money visual stun film...


In [102]:
from nltk.corpus import stopwords

nltk.download('stopwords')
", ".join(stopwords.words("english"))

STOPWORDS = set(stopwords.words("english"))

#exemplo de adição de palavras mais frequentes 
#STOPWORDS.add('movie')
#STOPWORDS.add('one')
#STOPWORDS.add('film')

def remove_stopwords(text):
    return " ".join([word for word in str(text).split() 
                     if word not in STOPWORDS])

df['review_stopwords'] = df['review_sem_pont']\
    .apply(lambda text : remove_stopwords(text))
df.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dimmy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,review,sentiment,review_lower,review_sem_pont,review_stopwords
0,One of the other reviewers has mentioned that ...,positive,one of the other reviewers has mentioned that ...,one of the other reviewers has mentioned that ...,one reviewers mentioned watching 1 oz episode ...
1,A wonderful little production. <br /><br />The...,positive,a wonderful little production. the filming tec...,a wonderful little production the filming tech...,wonderful little production filming technique ...
2,I thought this was a wonderful way to spend ti...,positive,i thought this was a wonderful way to spend ti...,i thought this was a wonderful way to spend ti...,thought wonderful way spend time hot summer we...
3,Basically there's a family where a little boy ...,negative,basically there's a family where a little boy ...,basically theres a family where a little boy j...,basically theres family little boy jake thinks...
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,"petter mattei's ""love in the time of money"" is...",petter matteis love in the time of money is a ...,petter matteis love time money visually stunni...


In [103]:
from collections import Counter
cnt = Counter()
for text in df['review_stopwords'].values:
    for word in text.split():
        cnt[word]+=1
        
cnt.most_common(10)



[('movie', 83506),
 ('film', 74462),
 ('one', 50366),
 ('like', 38825),
 ('good', 28483),
 ('even', 24280),
 ('would', 24001),
 ('time', 23265),
 ('really', 22894),
 ('see', 22432)]

In [104]:
def count_frequencia_por_linha(df, word):
    count = 0
    for text in df['review_stopwords'].values:
        if word in text:
            count+=1
    return count

count_frequencia_por_linha(df, "one")

38423

In [105]:
num_palavras_raras=10
PALAVRAS_RARAS = set([(word, c) for (word, c) 
                      in cnt.most_common()
                      [:-num_palavras_raras-1:-1]])
PALAVRAS_RARAS

{('bumi', 1),
 ('clatter', 1),
 ('frenchonly', 1),
 ('goodat', 1),
 ('horriblecatwoman', 1),
 ('originalif', 1),
 ('superbi', 1),
 ('terminalannie', 1),
 ('theirsall', 1),
 ('yosemitei', 1)}

In [106]:
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()

def stem_words(text):
    return " ".join([stemmer.stem(word) for word in text.split()])

frase_teste = "Inactive your powered beautiful balls"
print(stem_words(frase_teste))

df['review_stemmed'] = df['review_stopwords']\
    .apply(lambda text : stem_words(text))

df.head()

inact your power beauti ball


,review,sentiment,review_lower,review_sem_pont,review_stopwords,review_stemmed
0,One of the other reviewers has mentioned that ...,positive,one of the other reviewers has mentioned that ...,one of the other reviewers has mentioned that ...,one reviewers mentioned watching 1 oz episode ...,one review mention watch 1 oz episod youll hoo...
1,A wonderful little production. <br /><br />The...,positive,a wonderful little production. the filming tec...,a wonderful little production the filming tech...,wonderful little production filming technique ...,wonder littl product film techniqu unassum old...
2,I thought this was a wonderful way to spend ti...,positive,i thought this was a wonderful way to spend ti...,i thought this was a wonderful way to spend ti...,thought wonderful way spend time hot summer we...,thought wonder way spend time hot summer weeke...
3,Basically there's a family where a little boy ...,negative,basically there's a family where a little boy ...,basically theres a family where a little boy j...,basically theres family little boy jake thinks...,basic there famili littl boy jake think there ...
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,"petter mattei's ""love in the time of money"" is...",petter matteis love in the time of money is a ...,petter matteis love time money visually stunni...,petter mattei love time money visual stun film...


In [107]:
from nltk.stem.snowball import SnowballStemmer
#outras línguas que possue suporte para stemmer
SnowballStemmer.languages

('arabic',
 'danish',
 'dutch',
 'english',
 'finnish',
 'french',
 'german',
 'hungarian',
 'italian',
 'norwegian',
 'porter',
 'portuguese',
 'romanian',
 'russian',
 'spanish',
 'swedish')

In [108]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')
lemmatizer = WordNetLemmatizer()

def lemmatizer_words(text, lemma="v"):
    return " ".join([lemmatizer.lemmatize(word, lemma) 
                     for word in text.split()])

frase_teste = "Striper running walked"
print(lemmatizer_words(frase_teste, "v"))
print(lemmatizer_words(frase_teste, "n"))

Striper run walk
Striper running walked


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dimmy\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\dimmy\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [109]:
!pip install pyspellchecker
from spellchecker import SpellChecker
spell = SpellChecker()
def correct_spelling(text):
    correct_text = []
    miss_text = spell.unknown(text.split())
    if len(miss_text) == 0:
        return text
    for word in text.split():
        if word in miss_text:
            correct_text.append(spell.correction(word))
        else:
            correct_text.append(word)
    return " ".join(correct_text)
    
print(correct_spelling("speling rules"))

spelling rules
